In [ ]:
import pandas as pd
from sklearn.metrics import r2_score
from timeseriesmetrics import theil

def computeAccuracyModels(model_name, city_name, y_pred, y_true):
    """
    Calcula as métricas Theil e R² para um modelo e localidade e atualiza o DataFrame consolidado.
    Adiciona apenas as colunas para o modelo especificado, sem recriar as linhas de localidades já existentes.
    """
    global df_metrics  # DataFrame consolidado de métricas

    # Garantir que y_pred e y_true são arrays numpy ou pandas
    y_pred = y_pred.values if isinstance(y_pred, pd.Series) else y_pred
    y_true = y_true.values if isinstance(y_true, pd.Series) else y_true

    # Cálculo das métricas
    r2 = r2_score(y_true, y_pred)
    theil_stat = theil(y_true, y_pred)

    # Nome das colunas dinâmicas para este modelo
    r2_col_name = f'R2_{model_name}'
    theil_col_name = f'Theil_{model_name}'

    # Se a localidade já existe no DataFrame, adiciona/atualiza as colunas para o modelo
    if city_name in df_metrics['Localidade'].values:
        df_metrics.loc[df_metrics['Localidade'] == city_name, r2_col_name] = r2
        df_metrics.loc[df_metrics['Localidade'] == city_name, theil_col_name] = theil_stat
    else:
        # Caso a localidade não exista, levanta um erro
        raise ValueError(f"Localidade '{city_name}' não encontrada no DataFrame. Adicione-a antes de usar a função.")
    
    
    
    
import matplotlib.pyplot as plt

# Função para aplicar Random Walk em múltiplas séries temporais

def random_walk_forecast_multiple(series_dict):
    forecast_results = {}
    
    for city_name, series in series_dict.items():

        # Previsão usando Random Walk
        y_forecast = series.shift(1)
        y_forecast.iloc[0] = series.iloc[0]  # Ajuste o primeiro valor para evitar NaN
        
        # Armazenar as previsões separadas para cada cidade
        forecast_results[city_name] = y_forecast
    
    return forecast_results


# Função para calcular os lags significantes

from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

def getSignificantLags(y, nLags = 5, alpha=0.05):
    Pacf, Pacf_intervalos =  pacf(x=y, nlags=nLags, alpha=alpha)
   
    significantLags = []
    for i in range(1, len(Pacf)):
        # print(pac[i], pac_ci[i][0], pac_ci[i][1])
        if Pacf[i] < Pacf_intervalos[i][0] - Pacf[i] or Pacf[i] > Pacf_intervalos[i][1] - Pacf[i]:
            significantLags.append(i)
    print('Lags Significantes:', significantLags)
    return significantLags


# Processa os resíduos de previsão e identifica os lags mais relevantes.    

def process_residuals_print(y_true, y_forecast, city_name, nLags=5, alpha=0.07):
    """
    # Processa os resíduos de previsão e identifica os lags mais relevantes.
    
    Parâmetros:
    - y_true: Série original.
    - y_forecast: Previsões da série.
    - city_name: Nome da cidade.
    - nLags: Número máximo de lags a serem analisados (padrão: 5).
    - alpha: Nível de significância para cálculo do PACF (padrão: 0.05).
    
    Retorno:
    - city_name: Nome da cidade.
    - significantLags: Lista com os lags significativos.
    """
    # Calcular resíduos (erro entre a série original e a previsão)
    residuos_serie = pd.DataFrame({
        'residuos': y_true - y_forecast
    }, index=y_true.index)
    
    # Identificar lags mais relevantes
    significantLags = getSignificantLags(residuos_serie['residuos'], nLags=nLags, alpha=alpha)
    
    return city_name, significantLags    


# Função para calcular resíduos, variáveis lagged e escalonamento para cada cidade

def process_residuals(y_true, y_forecast):
    # Calcular resíduos (erro entre a série original e a previsão)
    residuos_serie = y_true - y_forecast
    # Construir variáveis lagged para previsão
    X_residual = residuos_serie.shift(1)  # Resíduo defasado em um ponto
    # Ajustar os valores iniciais ausentes para consistência
    X_residual.iloc[0] = X_residual.iloc[1]  # Ajustar o primeiro valor

    X_residual.columns = ['Lag_1']


    y_residual = residuos_serie.loc[X_residual.index]  # Valores dos resíduos

    return X_residual, y_residual



# Função para aplicar a busca Bayesiana e salvar os melhores parâmetros


from skopt import BayesSearchCV
from sklearn.model_selection import TimeSeriesSplit
from ridge_sgd_kernel import RidgeSGDKernelTorch
from ridge_sgd_kernel import RidgeSGDKernelTorch
from skopt import BayesSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np

class RidgeSGDKernelTorchWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, eta=0.01, c=0.01, sigma=1.0):
        self.eta = eta
        self.c = c
        self.sigma = sigma
        self.model = None

    def fit(self, X, y):
        # Inicializar o modelo
        self.model = RidgeSGDKernelTorch(eta=self.eta, c=self.c, sigma=self.sigma)
        # Treinar online
        for x_new, y_new in zip(X, y):
            self.model.partial_fit(x_new, y_new)
        return self

    def predict(self, X):
        # Realizar previsões
        return np.array([self.model.predict(x) for x in X])


def apply_bayes_search_for_city(city, X, y, param_grid):
    # Criar o modelo de busca Bayesiana
    bayes_search = BayesSearchCV(
        RidgeSGDKernelTorchWrapper(),
        search_spaces=param_grid,
        n_iter=30,
        cv=TimeSeriesSplit(n_splits=3),
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

    # Realizar a busca
    bayes_search.fit(X.values, y.values)

    # Obter os melhores parâmetros da busca
    best_params = bayes_search.best_params_

    # Criar e treinar o modelo com os melhores parâmetros encontrados
    ridge_sgd_torch_best = RidgeSGDKernelTorch(
        eta=best_params['eta'],
        c=best_params['c'],
        sigma=best_params['sigma']
    )

    # Lista para armazenar previsões
    y_pred_best = []

    # Treinar e prever de forma online com os dados
    for x_new, y_new in zip(X.values, y.values):
        ridge_sgd_torch_best.partial_fit(x_new, y_new)
        prediction = ridge_sgd_torch_best.predict(x_new)
        y_pred_best.append(prediction)

    # Previsões indexadas
    y_pred_bayes_indexed = pd.Series(y_pred_best, index=y.index)

    return y_pred_bayes_indexed, best_params


import matplotlib.pyplot as plt
import pandas as pd
from norm import RidgeSGDKernelTorchDict
import torch
def apply_model_and_plot(X_residual, y_residual, best_params):
    # Converter os dados em dicionários para serem compatíveis com o modelo
    X_residual_dict = {idx: x for idx, x in zip(y_residual.index, X_residual.values)}
    y_residual_dict = {idx: y for idx, y in zip(y_residual.index, y_residual.values)}

    # Instanciar o modelo com os melhores parâmetros encontrados para a cidade
    ridge_sgd_torch = RidgeSGDKernelTorchDict(eta=best_params['eta'], c=best_params['c'], sigma=best_params['sigma'])

    # Dicionário para armazenar previsões
    y_pred = {}

    # Treinar e prever de forma online com os dados residuais
    for idx in X_residual_dict.keys():
        x_new = X_residual_dict[idx]
        y_new = y_residual_dict[idx]

        # Atualizar o modelo com o novo ponto e obter a previsão diretamente
        pred_n = ridge_sgd_torch.partial_fit(x_new, y_new, idx)

        # Garantir que `pred_n` seja convertido corretamente para escalar
        if isinstance(pred_n, torch.Tensor):
            y_pred[idx] = pred_n.item()
        else:
            y_pred[idx] = pred_n

    # Converter as previsões em um pandas.Series para facilitar o plot
    y_pred_series = pd.Series(y_pred).sort_index()
    y_residual_series = pd.Series(y_residual_dict).sort_index()

    return y_residual_series, y_pred_series

